In [3]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
import logging

from actioncard import Chapel, Moat, Village, Militia, Moneylender, Smithy, CouncilRoom, Laboratory, Market, Witch
from aiutils import load, save
from config import GameConfig
from constants import ACTION, BUY
from enums import StartingSplit
from env import DefaultEnvironment, FullEnvironment
from player import load_players
from simulations import simulate

In [2]:
default_kingdom = [Chapel, Moat, Village, Militia, Moneylender, Smithy, CouncilRoom, Laboratory, Market, Witch]

In [4]:
project_root = '/home/justiny/Documents/Projects/PyDominion'
# project_root = '/home/justiny/Princeton/PyDominion'
model_dir = os.path.join(project_root, 'models')

In [5]:
logger = logging.getLogger()
logger.setLevel(logging.ERROR)
# logger.setLevel(BUY)
# logger.setLevel(ACTION)

## First-mover advantage ##

In [47]:
config = GameConfig(splits=[StartingSplit.Starting25Split, StartingSplit.Starting25Split], sandbox=False, num_players=2, must_include=default_kingdom)
players = load_players(['DW', 'DW'], None, map_location='cpu', train=False)
agent = players[0]
env = FullEnvironment(config, players, logger=logger)

In [48]:
n = 10000

In [49]:
sim_stats = simulate(env, n, None, turn_log=True, action_log=True)

100%|██████████| 10000/10000 [07:09<00:00, 23.27it/s]
===SUMMARY===
{0: 6052, 1: 4302, 'ProvinceWins': 10000, 'Ties': 354}


In [50]:
data_path = os.path.join(project_root, 'data', 'dw25-dw25-10k-stats')

In [51]:
save(data_path, sim_stats)

## Tree Simulations ##

In [22]:
from mcts import GameTree

In [54]:
tree_name = 'r-r-mcts-bm-hot-start-100k-srat-sandbox-ucb1-cdecay'
tree_path = os.path.join(model_dir, tree_name)
rollout_name = 'default-sandbox-mlog-100k'
rollout_path = os.path.join(model_dir, rollout_name)

In [9]:
tree_name = 'r-r-mcts-bm-warm-start-10k-srat-sandbox-ucb1'
rollout_name = 'default-sandbox-mlog-10k-v2'
tree_path = os.path.join(model_dir, tree_name)
rollout_path = os.path.join(model_dir, rollout_name)

In [24]:
tree = GameTree.load(tree_path, train=False, _skip_level=False, selection='max')

In [72]:
tree.selection = 'max'
tree.selection

'max'

In [13]:
tree.skip_level = True

In [55]:
# config = GameConfig(splits=[StartingSplit.Starting25Split, StartingSplit.Starting25Split], sandbox=True, num_players=2)
# splits = [StartingSplit.Starting25Split, StartingSplit.Starting25Split]
splits = [StartingSplit.StartingRandomSplit, StartingSplit.StartingRandomSplit]
config = GameConfig(splits=splits, sandbox=True, num_players=2, must_include=[])
players = load_players(['UCT', 'BM'], [rollout_path], tree=tree, train=False, rollout_type='mlog', use_tree=False)
# agent = load_players(['UCT'], [rollout_path], tree=tree, train=False, rollout_type='mlog', use_tree=False)[0]
# players = [agent, agent]
env = DefaultEnvironment(config, players, logger=logger)

In [47]:
logger.setLevel(logging.ERROR)
# logger.setLevel(BUY)

In [57]:
sim_data = simulate(env, 1000, tree, turn_log=False)

100%|██████████| 1000/1000 [00:16<00:00, 59.28it/s]===SUMMARY===
{0: 755, 1: 284, 'ProvinceWins': 998, 'Ties': 39}



In [97]:
data_dir = os.path.join(project_root, 'data')
data_name ='bm25-bm25-10k-scores'
data_path = os.path.join(data_dir, data_name)

In [98]:
save(data_path, sim_data.player_df)